Imports needed for to run this program. In particular, note that it needs both the $\texttt{pyvisa}$ and $\texttt{qcodes}$ packages. Has to be run from a python environment where these are installed.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyvisa
from qcodes import VisaInstrument
import time
from datetime import datetime
from os.path import exists
import logging
logging.basicConfig(level=logging.INFO)






This is the class that performs the measurement. Can be imported into other files, as long as the import requirements are fulfilled. $\newline$
Every measurement device has to be instantiated as an object of this class. The function cooldown then takes these and performs the measurement. Essentially, up to 4 instantiated multimeters take voltage periodic voltage readings in the interval $\texttt{interval}$, and save the data in a file. A plot showing this data is also created. $\newline$
$\textbf{Note:}$ While the most common errors of an open or already existing file should be caught by the function, it will display very strange behaviour if a device is unplugged and then plugged in again while a measurement is supposed to be running. In this case, restart both the multimeter and the kernel of the notebook, and then try again.

In [1]:
class DMM(VisaInstrument):
    def __init__(self,
                 name: str,
                 calib_file = None,
                 address: str="GPIB0::15::INSTR",
                 terminator: str="\n",
                 debug_mode: bool=False,
                 **kwargs):
        self.debug_mode = debug_mode

        super().__init__(name=name,
                         address=address,
                         terminator=terminator,
                         **kwargs)


        self.add_function('reset', call_cmd='*RST')
        

"""
Performs periodic voltage measurements.
Args: data_path (string): Path where the generated data will be saved
      Multimeter1 (DMM): First Multimeter that will take measurements, has to be given
      Multimeter2/3/4 (DMM): Any additional Multimeters that measure simultaneously, optional.
      range_ (int): Range setting for Multimeter in [Volt]
      resolution (float): Resolution setting for Multimeters, if possible should be small (0.001)
      saveint (int): Number of measurements taken before a batch of data is saved to the corresponding file
"""
        
def cooldown( data_path,Multimeter1, Multimeter2 = None, Multimeter3 = None, Multimeter4 = None, range_ = 2, resolution = 0.001, interval = 1, save_int = 10):

        start = f'{datetime.now()}'
        start = start[:10] + ',' + start[11:13] + '-' + start[14:16] + '-' +start[17:19]
        start_time = time.time()
        filename = data_path + 'cooldown_at' f'{start}' + '.csv'
        data = []
        
        times = []
        R1 = []
        R2 = []
        R3 = []
        R4 = []
        for i in range(3600*18):
            
        
            t = time.time() - start_time
            times.append(t)
            r1 = float(Multimeter1.ask(f'MEAS:VOLT:DC? {range_}, {resolution}'))
            R1.append(r1)
            if(Multimeter2):
                r2 = float(Multimeter2.ask(f'MEAS:VOLT:DC? {range_}, {resolution}'))
                R2.append(r2)
                if(Multimeter3):
                    r3 = float(Multimeter3.ask(f'MEAS:VOLT:DC? {range_}, {resolution}'))
                    R3.append(r3)
                    if(Multimeter4):
                        r4 = float(Multimeter4.ask(f'MEAS:VOLT:DC? {range_}, {resolution}'))
                        R4.append(r4)
            if(not i%10):
                if(Multimeter4):
                    data = np.column_stack([times,R1,R2,R3,R4])
                elif(Multimeter3):
                    data = np.column_stack([times,R1,R2,R3])
                elif(Multimeter2):
                    data = np.column_stack([times,R1,R2])
                else:
                    data = np.column_stack([times,R1])
                    
                try:
                    np.savetxt(filename, data)
                
                except IOError:
                    for i in range(5):
                        try:
                            np.savetxt(filename + f'CONTINUED:{i}', data)
                        except IOError:
                            print("Please close the file while measuring or check for wrong filename")
                            continue
                    break
                minutes = np.array(times)/60
                minutes = minutes.tolist()
                plt.figure()
                plt.grid(True)
                plt.plot(minutes,R1)
                plt.plot(minutes[:len(R2)],R2)
                plt.plot(minutes[:len(R3)],R3)
                plt.plot(minutes[:len(R4)],R4)
                plt.xlabel("Time [min]")
                plt.ylabel("Voltage [V]")
                plt.grid(True)
                plt.savefig(filename[:-4] + '.png')
                plt.close()
                print(datetime.now())

            time.sleep(interval)

NameError: name 'VisaInstrument' is not defined

Lists all devices available to a computer via $\texttt{pyvisa}$. Digital multimeters tend to have the format $\texttt{'USB0::0x...::0x....::something::INSTR'}$

In [3]:
rm = pyvisa.ResourceManager()
list_re = rm.list_resources()
print(list_re)



('USB0::0x0957::0x1A07::MY53200916::INSTR',)


Instantiate the digital multimeters, using the addresses displayed by the previous section.
If a name is already used, no new object will be instantiated, but the older instantiation of an object with this name will remain valid. $\newline$
The first object is pinged to make sure the connections work. Sometimes when plugging multimeters in and out instantiation might not directly all out an error, but the ping will fail.  

In [4]:
name1 = 'Harry'
name2 = 'Ron'
name3 = 'Hermione'

try:
    Multimeter1 = DMM(name1,address = 'USB0::0x0957::0x1A07::MY53200916::0::INSTR')
   # Multimeter2 = DMM(name1,address2)
    #Multimeter3 = DMM(name1,address3)
    
    Multimeter1.ask("*IDN?")
    
    logging.info(f'Initialized a Multimemter {name1} successfully')
except KeyError:
    logstring = "The name you chose is already used! \n"
    logstring += "You can either just use Multimeter1 and jump to the next cell, or give it a new name"
    logging.error(logstring)


INFO:qcodes.instrument.base.com.visa:[Harry4(DMM)] Opening PyVISA Resource Manager with default backend.
INFO:qcodes.instrument.base.com.visa:[Harry4(DMM)] Opening PyVISA resource at address: USB0::0x0957::0x1A07::MY53200916::0::INSTR
INFO:root:Initialized a Multimemter Harry4 successfully


Choose a path for saving the file, and call the measurement function.

In [5]:
data_path = 'C:/Users/Hansd/Documents/Berkelius/Data/Electron'

# Here we finally call the cooldown

cooldown(data_path,Multimeter1,range_ = 1, resolution = 0.01, interval = 1)

2022-09-22 18:38:17.423680


KeyboardInterrupt: 